In [2]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime

In [11]:
class CrawlStock3PRatioTW:
    def __init__(self, date):
        self.date = date
        self.date_str = date.strftime("%Y%m%d")
        self.target_name = "台股3p數據"
        self.sub_market = ["sii", "otc"]

    def crawl_sii(self):
        r= requests.get('http://www.twse.com.tw/exchangeReport/BWIBBU_d?response=csv&date='+self.date_str+'&selectType=ALL')
        content = r.text.replace('=', '')
        lines = content.split('\n')
        lines = list(filter(lambda l:len(l.split('",')) > 5, lines))
        content = "\n".join(lines[:])
        if content == '':
            return None
        df= pd.read_csv(StringIO(content), header=0).dropna(how='all',axis=1)
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(',', ''))
        df=df[['證券代號','證券名稱','本益比','殖利率(%)','股價淨值比']]
        df.iloc[:,2:]=df.iloc[:,2:].apply(lambda s:pd.to_numeric(s, errors='coerce'))
        df=df.rename(columns={k:v for k,v in zip(df.columns,['stock_id','stock_name','dividend_yield','pe','pb'])})
        df['date']=self.date
        return df


    def crawl_otc(self):
        y = str(int(self.date.strftime("%Y")) - 1911)
        date_str = y + "/" + self.date.strftime("%m") + "/" + self.date.strftime("%d")
        link = 'https://www.tpex.org.tw/web/stock/aftertrading/peratio_analysis/pera_download.php?l=zh-tw&d='+date_str+'&s=0,asc,0'
        r = requests.get(link)
        lines = r.text.replace('\r', '').split('\n')
        if len(lines)<10:
            return None
        df = pd.read_csv(StringIO("\n".join(lines[3:-1])), header=0)
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(',', ''))
        df=df.drop(columns=['每股股利','股利年度'])
        df=df.rename(columns={k:v for k,v in zip(df.columns,['stock_id','stock_name','pe','dividend_yield','pb'])})
        df.iloc[:,2:]=df.iloc[:,2:].apply(lambda s:pd.to_numeric(s, errors='coerce'))
        df['date']=self.date
        return df

    def crawl_main(self):
        try:
            df = pd.concat([self.crawl_sii(), self.crawl_otc()])
        except ValueError:
            return None
        return df
    
z=CrawlStock3PRatioTW(datetime.datetime(2009,1,5)).crawl_main()
z

,stock_id,stock_name,dividend_yield,pe,pb,date
0,1101,台泥,15.45,7.23,1.3,2009-01-05
1,1102,亞泥,8.98,10.64,1.25,2009-01-05
2,1103,嘉泥,13.91,7.81,0.61,2009-01-05
3,1104,環泥,17.74,6.18,0.68,2009-01-05
4,1108,幸福,17.29,4.13,0.57,2009-01-05
...,...,...,...,...,...,...
527,9949,琉園,8.05,19.83,1.21,2009-01-05
528,9950,萬國通,9.62,10.72,0.7,2009-01-05
529,9951,皇田,19.66,3.39,0.52,2009-01-05
530,9960,邁達康,20.22,5.49,0.74,2009-01-05
